<a href="https://colab.research.google.com/github/RafsanJany-44/Learning_MNE/blob/main/Haaglanden_Medisch_Centrum_sleep_staging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Dataset](https://physionet.org/content/hmc-sleep-staging/1.0.0/)

In [1]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.8 MB/s eta 0:00:00


In [ ]:
!wget -r -N -c -np https://physionet.org/files/hmc-sleep-staging/1.0.0/

In [3]:
path_root = '/content/physionet.org/files/hmc-sleep-staging/1.0.0/recordings'

In [4]:
import os
os.chdir(path_root)

In [5]:
import os
os.getcwd()

'/content/physionet.org/files/hmc-sleep-staging/1.0.0/recordings'

In [6]:
import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.datasets.sleep_physionet.age import fetch_data

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from glob import glob

In [8]:
data=glob(path_root + '/*.edf')
print(len(data))

data_eeg = [i for i in data if 'S' and not 'sleepscoring' in i.split('/content/physionet.org/files/hmc-sleep-staging/1.0.0/recordings')[1]]
data_ano = [i for i in data if 'S'and 'sleepscoring' in i.split('/content/physionet.org/files/hmc-sleep-staging/1.0.0/recordings')[1]]

23


In [9]:
print(len(data_eeg),len(data_ano))

12 11


**Creating a dictionnary with two columns: EEG files and Annotation files**

In [10]:
dictionary={}
for i,eeg in enumerate(data_eeg):
  dictionary[i+1]=[eeg]
for i,ano in enumerate(data_ano):
  dictionary[i+1].append(ano)

In [11]:
dictionary[1]

['/content/physionet.org/files/hmc-sleep-staging/1.0.0/recordings/SN010.edf',
 '/content/physionet.org/files/hmc-sleep-staging/1.0.0/recordings/SN008_sleepscoring.edf']

**Converting the created dictionnary into DataFrame**

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth',200)
dt = pd.DataFrame.from_dict(dictionary)
dt=dt.transpose()
dt.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test=train_test_split(dt,test_size=0.15)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train.iloc[:,0]

In [ ]:
X_train.iloc[3,1]

In [ ]:
#YAHYA, BELLO = 0, 1
#[yahya_files, bello_files] = dictionary[1], dictionary[2]
raw_train=[]
annot_train=[]
for i in X_train.iloc[:,0].tolist():
   raw_train.append(mne.io.read_raw_edf(str(i),stim_channel='Event marker',
                                misc=['Temp rectal']))
for i,k in enumerate(X_train.iloc[:,1].tolist()):
   annot_train.append(mne.read_annotations(str(k)))
   raw_train[i].set_annotations(annot_train[i], emit_warning=False)
   raw_train[i].load_data()

# plot some data
# scalings were chosen manually to allow for simultaneous visualization of
# different channel types in this specific dataset
# raw_train.plot(start=10, duration=10,
#                scalings=dict(eeg=1e-4, resp=1e3, eog=1e-4, emg=1e-7,
#                              misc=1e-1))

In [ ]:
%matplotlib inline

In [ ]:
raw_train[1].plot(start=10, duration=10,
               scalings=dict(eeg=1e-4, resp=1e3, eog=1e-4, emg=1e-7,
                            misc=1e-1))


**Preprocessing of data: Filtering and ICA**

In [ ]:
filt_raw=[]
for i in raw_train:
  filt_raw.append(i.copy().filter(l_freq=1., h_freq=None))